In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import geocoder # import geocoder

page = requests.get(" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code
page.content

soup = BeautifulSoup(page.content, 'lxml')
  
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
#print(df[0].to_json(orient='records'))
df = df[0]
df.head()


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [3]:
##cleaning data - removing Not assigned boroughs and replacing not assigned neighbourhoods with boroughs.
df.loc[0,:]
df.columns = df.loc[0,:]#['a', 'b']
df.drop([0],axis=0,inplace = True)
df = df[df['Borough']!='Not assigned']  # removing entries with no value for Borough
df = df.reset_index()                   #resetting index after removing
df.drop(['index'],axis=1,inplace = True)
temp = df[df['Neighbourhood']=='Not assigned'].index.tolist() # getting a list of Neighbourhoods with the value "Not assigned."
df.loc[temp,'Neighbourhood'] = df.loc[temp,'Borough'] ## replacing those neighbourhoods with borough name

df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [9]:
#finding and grouping Neighborhoods with the same Post code
for index, row in df.iterrows():
      row['Neighbourhood'] =  df[df['Postcode']== row['Postcode']]['Neighbourhood'].str.cat(sep=',')

        
df.drop_duplicates(subset='Postcode', keep='first', inplace=True) #dropping the duplicates
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [6]:
##geocoder is taking forever to respond. Therefore, continuing with the provided location data.
geodata = pd.read_csv('https://cocl.us/Geospatial_data')
geodata.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
#For each row in geodata frame, check the postal code and add Latitude and Longitude values to the df frame.

for index, row in geodata.iterrows():
    temp = df[df['Postcode'] == row['Postal Code']].index.tolist()
    df.loc[temp,'Latitude'] = row['Latitude']
    df.loc[temp,'Longitude'] = row['Longitude']

In [10]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
4,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
6,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
7,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
8,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
10,M3B,North York,Don Mills North,43.745906,-79.352188
11,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
13,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


In [11]:
df.shape

(103, 5)